### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [1]:
# Write your code from here
# 1. Create a .proto file for gRPC service (data_validation.proto):
#
# syntax = "proto3";
#
# package data_validation;
#
# message UserRequest {
#   string user_id = 1;
#   string name = 2;
#   int32 age = 3;
#   string email = 4;
# }
#
# message ValidationResponse {
#   bool is_valid = 1;
#   string message = 2;
# }
#
# service DataValidationService {
#   rpc ValidateUser(UserRequest) returns (ValidationResponse);
# }

# 2. Generate the gRPC and protobuf files:
#
# Ensure you have the following installed:
#   - Python
#   - grpcio (pip install grpcio)
#   - protobuf (pip install protobuf)
#   - grpcio-tools (pip install grpcio-tools)
#
# Run the following command in your terminal, in the directory where you saved data_validation.proto:
#
# python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. data_validation.proto
#
# This will generate two Python files:
#   - data_validation_pb2.py
#   - data_validation_pb2_grpc.py
#
# These generated files contain the code for the protobuf messages and the gRPC service,
# and they need to be in the same directory as your server.py script.

# 3. Implement server-side validation (server.py):
import grpc
from concurrent import futures
import data_validation_pb2  # Import the generated protobuf module
import data_validation_pb2_grpc # Import the generated gRPC module
import re

class DataValidationService(data_validation_pb2_grpc.DataValidationServiceServicer):
    def ValidateUser(self, request, context):
        """
        Validates user data according to specified rules.
        """
        user_id = request.user_id
        name = request.name
        age = request.age
        email = request.email

        # Validation rules
        if not user_id:
            return data_validation_pb2.ValidationResponse(
                is_valid=False, message="User ID is required"
            )
        if not name:
            return data_validation_pb2.ValidationResponse(
                is_valid=False, message="Name is required"
            )
        if not (1 <= age <= 150):
            return data_validation_pb2.ValidationResponse(
                is_valid=False, message="Age must be between 1 and 150"
            )
        if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
            return data_validation_pb2.ValidationResponse(
                is_valid=False, message="Invalid email format"
            )

        return data_validation_pb2.ValidationResponse(
            is_valid=True, message="User data is valid"
        )

def serve():
    """
    Starts the gRPC server.
    """
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    data_validation_pb2_grpc.add_DataValidationServiceServicer_to_server(
        DataValidationService(), server
    )
    server.add_insecure_port("[::]:50051")  # Listen on all interfaces on port 50051
    server.start()
    print("Server started, listening on [::]:50051")
    try:
        server.wait_for_termination()
    except KeyboardInterrupt:
        print("Server stopped.")

if __name__ == "__main__":
    serve()



ModuleNotFoundError: No module named 'data_validation_pb2'